In [1]:
import pandas as pd
import numpy as np
import os
import plotly.express as px
import plotly.graph_objects as go


In [2]:
file_dir = 'D:/Astro/StarCatalogues'
file_name = 'Imm Deep Sky Compendium - 2024 - rev5a.xlsm'
file_path = os.path.join(file_dir, file_name)


In [3]:
data = pd.read_excel(file_path, sheet_name="Main", skiprows=6)


c:\Users\Nina\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [4]:
# Fix up the column headers

# Extract column names for columns 0 to 33, and from row 0 for columns 34 onward
header = list(data.columns[:34]) + data.iloc[0, 34:].tolist()
substitutions = {
    'Object Name & Image': 'Object',
    'Unnamed: 1': 'Simbad',
    'Unnamed: 2': 'Aladin',
    'Right Ascension': 'RA_hms',
    'Unnamed: 12': 'RA_deg',
    'Declination': 'Dec_dms',
    'Unnamed: 14': 'Dec_deg',
    'Const.': 'Constellation',
    'Nick.': 'Name',
    'Alt. ID': 'Name_Alt',
    'Surf.': 'Brightness',
    'Inclin.': 'Inclination',
    17: 'Next17',
    19: 'Next19',
    21: 'Next21',
    23: 'Next23',
    'Unnamed': 'Rename',
}
header[-6]='ObjAz'
header[-5]='Key'
header[-4]='FOV'
header = [substitutions.get(col, col) for col in header]
index = [col_idx for col_idx, col_name in enumerate(header) if col_name == 'Abell']
header[index[0]]='AbellG'   # Replace duplate Abell with AbellG
header[index[1]]='AbellN'   # Replace duplate Abell with AbellG
data.columns = header


In [5]:
data

,Object,Simbad,Aladin,Type,Sub,Class,Size,Distance,Diameter,Rating,...,OB,SP,FG,HD,ObjAz,Key,FOV,Transit Time,Add. Filters,Sort to
0,Astrobin,SIMBAD,ALADIN,NaN,NaN,NaN,NaN,(mly or ly),(kly or ly),5 - High,...,OB,SP,FG,HD,NaN,ALADIN,ALADIN,Transit Time,Add. Filters,Sort to
1,Link,Link,Link,NaN,NaN,NaN,('),NaN,NaN,0 - Low,...,109,109,109,NaN,NaN,KEY,FOV,Sort,NaN,v4g
2,Sh2-139,S,A,Neb,Em,HII,11,9500,30,1,...,NaN,NaN,NaN,NaN,NaN,Sh2-139,0.366667,NaN,NaN,2747
3,NGC 7309,S,A,Gal,Spiral,Sc,1.8,190,100,2,...,NaN,NaN,NaN,NaN,NaN,NGC 7309,0.2,NaN,u,2507
4,V0375 Lac,S,A,Stars,YSO,NaN,3,2000,1.8,2,...,NaN,NaN,NaN,NaN,NaN,V0375 Lac,0.2,NaN,NaN,2958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3285,IAU:604,NaN,NaN,Moon,Saturnian Moon,NaN,0.042,NaN,NaN,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3286,IAU:603,NaN,NaN,Moon,Saturnian Moon,NaN,0.036,NaN,NaN,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3287,IAU:602,NaN,NaN,Moon,Saturnian Moon,NaN,0.03,NaN,NaN,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3288,IAU:601,NaN,NaN,Moon,Saturnian Moon,NaN,0.024,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Drop junk columns and first 2 rows (header data)
columns_to_drop = ['Simbad', 'Aladin', 'My', 'My.1', 'My.2', 'Next17', 'Next19', 'Next21', 'Next23', 'Time', 'Date', 'Alt.', 'Separ.', 'Index', 'ObjAz', 'Transit Time', 'Add. Filters', 'Sort to ']
data = data.drop(columns=columns_to_drop)
data = data.iloc[2:].reset_index(drop=True)  


In [7]:
data['RA_deg'] = pd.to_numeric(data['RA_deg'], errors='coerce')
data['RA_hr'] = pd.to_numeric(data['RA_deg']/15, errors='coerce')
data['Dec_deg'] = pd.to_numeric(data['Dec_deg'], errors='coerce')
data['Size'] = pd.to_numeric(data['Size'], errors='coerce')
data['Visual'] = pd.to_numeric(data['Visual'], errors='coerce')

In [8]:
data['NGC'] = pd.to_numeric(data['NGC'], errors='coerce').astype('Int64')
data['IC'] = pd.to_numeric(data['IC'], errors='coerce').astype('Int64')
data['H400'] = pd.to_numeric(data['H400'], errors='coerce').astype('Int64')
data['Messier'] = pd.to_numeric(data['Messier'], errors='coerce').astype('Int64')
data['Caldwell'] = pd.to_numeric(data['Caldwell'], errors='coerce').astype('Int64')
data['Arp'] = pd.to_numeric(data['Arp'], errors='coerce').astype('Int64')
data['Hickson'] = pd.to_numeric(data['Hickson'], errors='coerce').astype('Int64')
data['AbellG'] = pd.to_numeric(data['AbellG'], errors='coerce').astype('Int64')
data['AbellN'] = pd.to_numeric(data['AbellN'], errors='coerce').astype('Int64')
data['UGC'] = pd.to_numeric(data['UGC'], errors='coerce').astype('Int64')
data['PGC'] = pd.to_numeric(data['PGC'], errors='coerce').astype('Int64')
data['Griff.'] = pd.to_numeric(data['Griff.'], errors='coerce').astype('Int64')
data['Kohou.'] = pd.to_numeric(data['Kohou.'], errors='coerce').astype('Int64')
data['Mink.'] = pd.to_numeric(data['Mink.'], errors='coerce').astype('Int64')
data['Barn.'] = pd.to_numeric(data['Barn.'], errors='coerce').round().astype('Int64') # has decimals
data['Gum'] = pd.to_numeric(data['Gum'], errors='coerce').astype('Int64')
data['LBN'] = pd.to_numeric(data['LBN'], errors='coerce').astype('Int64')
data['LDN'] = pd.to_numeric(data['LDN'], errors='coerce').astype('Int64')
data['Sh2'] = pd.to_numeric(data['Sh2'], errors='coerce').astype('Int64')
#data['SNR'] = pd.to_numeric(data['SNR'], errors='coerce').astype('Int64')  # has text
data['vdB'] = pd.to_numeric(data['vdB'], errors='coerce').astype('Int64')
data['HT'] = pd.to_numeric(data['HT'], errors='coerce').astype('Int64')
data['SD'] = pd.to_numeric(data['SD'], errors='coerce').astype('Int64')
data['OB'] = pd.to_numeric(data['OB'], errors='coerce').astype('Int64')
data['SP'] = pd.to_numeric(data['SP'], errors='coerce').astype('Int64')
data['FG'] = pd.to_numeric(data['FG'], errors='coerce').astype('Int64')
data['HD'] = pd.to_numeric(data['HD'], errors='coerce').astype('Int64')

In [9]:
data.describe()

,Size,RA_deg,Dec_deg,Visual,NGC,IC,H400,Messier,Caldwell,Arp,...,LDN,Sh2,vdB,HT,SD,OB,SP,FG,HD,RA_hr
count,3168.000000,3264.000000,3264.000000,2103.000000,1445.0,178.0,400.0,110.0,109.0,338.0,...,32.0,313.0,159.0,109.0,108.0,109.0,109.0,109.0,118.0,3264.000000
mean,18.341894,185.652017,11.673504,11.099891,4152.653287,2139.719101,200.5,55.5,55.0,169.5,...,1096.25,157.0,80.0,55.0,54.962963,55.0,55.0,55.0,99684.338983,12.376801
std,69.122487,103.086117,34.376693,3.716283,2256.267236,1696.358196,115.614301,31.898276,31.609598,97.716426,...,448.020665,90.49954,46.043458,31.609598,31.754586,31.609598,31.609598,31.609598,62544.193555,6.872408
min,0.002400,0.000000,-90.000000,-26.740000,1.0,10.0,1.0,1.0,1.0,1.0,...,43.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,358.0,0.000000
25%,2.000000,92.093750,-13.984792,9.900000,2301.0,621.25,100.75,28.25,28.0,85.25,...,802.0,79.0,40.5,28.0,27.75,28.0,28.0,28.0,39028.5,6.139583
50%,4.500000,188.604658,11.907917,11.900000,4244.0,1555.0,200.5,55.5,55.0,169.5,...,1243.0,157.0,80.0,55.0,54.5,55.0,55.0,55.0,107436.5,12.573644
75%,13.000000,274.668750,39.029167,13.350000,6281.0,3846.25,300.25,82.75,82.0,253.75,...,1376.75,235.0,119.5,82.0,82.25,82.0,82.0,82.0,149531.75,18.311250
max,2400.000000,359.975000,90.000000,21.000000,7838.0,5378.0,400.0,110.0,109.0,338.0,...,1660.0,313.0,159.0,109.0,109.0,109.0,109.0,109.0,218045.0,23.998333


In [10]:
data

,Object,Type,Sub,Class,Size,Distance,Diameter,Rating,Notes,RA_hms,...,vdB,HT,SD,OB,SP,FG,HD,Key,FOV,RA_hr
0,Sh2-139,Neb,Em,HII,11.000,9500,30,1,Faint HII region with small brighter arc,223400,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Sh2-139,0.366667,22.566667
1,NGC 7309,Gal,Spiral,Sc,1.800,190,100,2,Unusual arm structure - 3 primary arms and 3 s...,223420,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NGC 7309,0.2,22.572222
2,V0375 Lac,Stars,YSO,NaN,3.000,2000,1.8,2,YSO that is the source of the tiny intricate w...,223441,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,V0375 Lac,0.2,22.578056
3,Abell 80,Neb,PN,EHY/q,2.200,6000,3.8,2,HII rim with faint OIII inner region. Shape is...,223446,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,PN A66 80,0.2,22.579444
4,Barnard 370,Neb,Dark,0,45.000,u,u,0,Very faint - Region of narrow dark lanes,223450,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Barnard 370,1.5,22.580556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3283,IAU:604,Moon,Saturnian Moon,NaN,0.042,NaN,NaN,2,Bright icy surface; visible in large scopes.,NaN,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN
3284,IAU:603,Moon,Saturnian Moon,NaN,0.036,NaN,NaN,2,Small and faint; requires dark skies.,NaN,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN
3285,IAU:602,Moon,Saturnian Moon,NaN,0.030,NaN,NaN,2,Bright icy moon with geysers; very faint.,NaN,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN
3286,IAU:601,Moon,Saturnian Moon,NaN,0.024,NaN,NaN,1,Small Saturnian moon with a large crater; fain...,NaN,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN


In [11]:
# Clean up the type column
data['Type'] = data['Type'].replace({'Neb ':'Neb'})
type_abbr = {
    'Neb': 'Nebula',
    'Gal': 'Galaxy',
    'Stars': 'Cluster',
    'Star': 'Star',
    'Planet': 'Planet',
    'Moon': 'Moon',
}
type_enum = {abbr: idx for idx, abbr in enumerate(type_abbr.keys())}
C1_lookup = {idx: name for idx, name in enumerate(type_abbr.values())}
data['C1'] = data['Type'].map(type_enum)
data['C1Name'] = data['C1'].map(C1_lookup)

unique_abbr = set(data['Type'].unique())
valid_abbr = set(type_abbr.keys())
# Find any values not in the lookup
invalid_abbr = unique_abbr - valid_abbr
missing_abbr = valid_abbr - unique_abbr

if invalid_abbr:
    print(f"Invalid or unmapped abbreviations found: {sorted(invalid_abbr)}")
elif missing_abbr:
    print(f"Missing abbreviations found: {sorted(missing_abbr)}")
else:
    print("All abbreviations are valid and mapped.")




All abbreviations are valid and mapped.


In [12]:
# Clean up the constellation column
data['Constellation'] = data['Constellation'].replace({'Apu':'Aps', 'Lmi': 'LMi', 'Uma': 'UMa', 'Crux': 'Cru', 'Cent': 'Cen', 'Pis': 'Pic'})
constellation_abbr = {
    'And': 'Andromeda', 'Ant': 'Antlia', 'Aps': 'Apus', 'Aql': 'Aquila', 'Aqr': 'Aquarius',
    'Ara': 'Ara', 'Ari': 'Aries', 'Aur': 'Auriga', 'Boo': 'Boötes', 'CMa': 'Canis Major',
    'CMi': 'Canis Minor', 'CVn': 'Canes Venatici', 'Cam': 'Camelopardalis', 'Cap': 'Capricornus',
    'Car': 'Carina', 'Cas': 'Cassiopeia', 'Cen': 'Centaurus', 'Cep': 'Cepheus', 'Cet': 'Cetus',
    'Cha': 'Chamaeleon', 'Cir': 'Circinus', 'Cnc': 'Cancer', 'Col': 'Columba', 'Com': 'Coma Berenices',
    'CrA': 'Corona Australis', 'CrB': 'Corona Borealis', 'Crt': 'Crater', 'Cru': 'Crux',
    'Crv': 'Corvus', 'Cyg': 'Cygnus', 'Del': 'Delphinus', 'Dor': 'Dorado', 'Dra': 'Draco',
    'Eri': 'Eridanus', 'For': 'Fornax', 'Gem': 'Gemini', 'Gru': 'Grus', 'Her': 'Hercules',
    'Hor': 'Horologium', 'Hya': 'Hydra', 'LMi': 'Leo Minor', 'Lac': 'Lacerta', 'Leo': 'Leo',
    'Lep': 'Lepus', 'Lib': 'Libra', 'Lup': 'Lupus', 'Lyn': 'Lynx', 'Lyr': 'Lyra',
    'Men': 'Mensa', 'Mic': 'Microscopium', 'Mon': 'Monoceros', 'Mus': 'Musca', 'Nor': 'Norma',
    'Oct': 'Octans', 'Oph': 'Ophiuchus', 'Ori': 'Orion', 'Pav': 'Pavo', 'Peg': 'Pegasus',
    'Per': 'Perseus', 'Phe': 'Phoenix', 'Pic': 'Pictor', 'PsA': 'Piscis Austrinus', 'Psc': 'Pisces', 'Pup': 'Puppis',
    'Pyx': 'Pyxis', 'Ret': 'Reticulum', 'Scl': 'Sculptor', 'Sco': 'Scorpius', 'Sct': 'Scutum',
    'Ser': 'Serpens', 'Sex': 'Sextans', 'Sge': 'Sagitta', 'Sgr': 'Sagittarius', 'Tau': 'Taurus',
    'Tel': 'Telescopium', 'TrA': 'Triangulum Australe', 'Tri': 'Triangulum', 'Tuc': 'Tucana',
    'UMa': 'Ursa Major', 'UMi': 'Ursa Minor', 'Vel': 'Vela', 'Vir': 'Virgo', 'Vol': 'Volans',
    'Vul': 'Vulpecula', 'Orb': 'Orbit'
}
const_enum = {abbr: idx for idx, abbr in enumerate(constellation_abbr.keys())}
Cn_lookup = {idx: name for idx, name in enumerate(constellation_abbr.values())}
data['Cn'] = data['Constellation'].map(const_enum)
data['CnName'] = data['Cn'].map(Cn_lookup)



unique_abbr = set(data['Constellation'].unique())
valid_abbr = set(constellation_abbr.keys())
# Find any values not in the lookup
invalid_abbr = unique_abbr - valid_abbr
missing_abbr = valid_abbr - unique_abbr

if invalid_abbr:
    print(f"Invalid or unmapped abbreviations found: {sorted(invalid_abbr)}")
elif missing_abbr:
    print(f"Missing abbreviations found: {sorted(missing_abbr)}")
else:
    print("All abbreviations are valid and mapped.")
data['Constellation'].unique()

data['ConstName'] = data['Constellation'].map(constellation_abbr)

grouped = data.groupby(['Constellation', 'Type'], observed=True).size().reset_index(name='Count')
fig = px.bar(grouped, x='Constellation', y='Count', color='Type', title='Object Constellation by Type', 
             category_orders={'Constellation': list(constellation_abbr.keys())} ) 
fig.show()



All abbreviations are valid and mapped.


In [13]:
# Clean up the type column
data['Sub'] = data['Sub'].replace({'Group ':'Group', 'Pair ': 'Pair', 'Spiral ': 'Spiral', 'Mol CLd':'Mol Cld', 'Stars':'Star' })
sub_abbr = {
    # Multiple Galaxies
    'Chain': 'Set of Chained Galaxies',
    'Cluster': 'Set of Clustered Galaxies',
    'Group': 'Set of Grouped Galaxies',
    'Merger': 'Set of Merging Galaxies',
    'Pair': 'Pair of Galaxies',
    'Trio': 'Trio of Galaxies',

    # Individual Galaxy
    'BCD': 'Blue Compact Dwarf Galaxy',
    'Coll': 'Collisional Ring Galaxy',
    'Dwarf': 'Dwarf Galaxy',
    'Ellip': 'Elliptical Galaxy',
    'Floc': 'Flocculent Galaxy',
    'Lent': 'Lenticular Galaxy',
    'Mag': 'Magellanic Galaxy',
    'Polar': 'Polar Galaxy',
    'Spiral': 'Spiral Galaxy',

    # Nebula
    'Dark': 'Dark Nebula',
    'Em': 'Emission Nebula',
    'Mol Cld': 'Molecular Cloud Nebula',
    'PN': 'Planetary Nebula',
    'PPN': 'Protoplanetary Nebula',
    'Refl': 'Reflection Nebula',
    'SNR': 'Supernova Remnant Nebula',

    # Stellar associations
    'GC': 'Globular Cluster',
    'HH': 'Herbig-Haro Object',
    'Nova': 'Nova Object',
    'OC': 'Open Cluster',
    'Star': 'Star',
    'Star Cld': 'Star Cloud',
    'YSO': 'Young Stellar Object',

    # Orbitals
    'Planet': 'Planet',
    'Dwarf Planet': 'Dwarf Planet',
    'Martian Moon': 'Martian Moon',
    'Galilean Moon': 'Galilean Moon',
    'Saturnian Moon': 'Saturnian Moon',
    'Martian Moon': 'Martian Moon',
    'Natural Satellite': 'Natural Satellite',
    'Space Station': 'Space Station',
    'Satellite': 'Satellite',
    'Rocket Body': 'Rocket Body',
    'Space Debris': 'Space Debris',
    'Comet': 'Comet',
    'Asteroid': 'Asteroid',
}
sub_enum = {abbr: idx for idx, abbr in enumerate(sub_abbr.keys())}
C2_lookup = {idx: name for idx, name in enumerate(sub_abbr.values())}
data['C2'] = data['Sub'].map(sub_enum)
data['C2Name'] = data['C2'].map(C2_lookup)

unique_abbr = set(data['Sub'].unique())
valid_abbr = set(sub_abbr.keys())
# Find any values not in the lookup
invalid_abbr = unique_abbr - valid_abbr
missing_abbr = valid_abbr - unique_abbr

if invalid_abbr:
    print(f"Invalid or unmapped abbreviations found: {sorted(invalid_abbr)}")
elif missing_abbr:
    print(f"Missing abbreviations found: {sorted(missing_abbr)}")
else:
    print("All abbreviations are valid and mapped.")


data['SubName'] = data['Sub'].map(sub_abbr)


grouped = data.groupby(['Sub', 'Type'], observed=True).size().reset_index(name='Count')
fig = px.bar(grouped, x='Sub', y='Count', color='Type', title='Object Subtype by Type') 
fig.show()



Missing abbreviations found: ['Asteroid', 'Comet', 'Rocket Body', 'Satellite', 'Space Debris', 'Space Station']


In [14]:
bins = [    0,         2,                4,                  6,                 8,              10,                  12,           3000]
labels = ['Brilliant (Mag <2)', 'Bright (Mag 2-4)', 'Visible (Mag 4-6)', 'Dim (Mag 6-8)', 'Faint (Mag 8-10)', 'Ghostly (Mag 10-12)', 'Ultra Faint (Mag 12+)']
data['VisualBin'] = pd.cut(data['Visual'], bins=bins, labels=labels, include_lowest=True)

reversed_labels =  labels[::-1] + ['Unknown']
Vz_enum = {abbr: idx for idx, abbr in enumerate(reversed_labels)}
Vz_lookup = {idx: name for idx, name in enumerate(reversed_labels)}
data['Vz'] = data['VisualBin'].map(Vz_enum).astype('Int64')
data['Vz'] = data['Vz'].fillna(len(reversed_labels)-1)
data['VzName'] = data['Vz'].map(Vz_lookup)


# Step 2: Group by SizeBin and Type
grouped = data.groupby(['Vz', 'Type'], observed=True).size().reset_index(name='Count')
grouped['VzName'] = grouped['Vz'].map(Vz_lookup)

# Step 3: Plot with color by Type
fig = px.bar(grouped, x='VzName', y='Count', color='Type',
             title='Object Apparent Mag Distribution by Type',
             category_orders={'VisualBin': labels})  # ensures correct bin order

fig.show()

In [15]:
data


,Object,Type,Sub,Class,Size,Distance,Diameter,Rating,Notes,RA_hms,...,C1Name,Cn,CnName,ConstName,C2,C2Name,SubName,VisualBin,Vz,VzName
0,Sh2-139,Neb,Em,HII,11.000,9500,30,1,Faint HII region with small brighter arc,223400,...,Nebula,17,Cepheus,Cepheus,16,Emission Nebula,Emission Nebula,NaN,7,Unknown
1,NGC 7309,Gal,Spiral,Sc,1.800,190,100,2,Unusual arm structure - 3 primary arms and 3 s...,223420,...,Galaxy,3,Aquila,Aquila,14,Spiral Galaxy,Spiral Galaxy,Ultra Faint (Mag 12+),0,Ultra Faint (Mag 12+)
2,V0375 Lac,Stars,YSO,NaN,3.000,2000,1.8,2,YSO that is the source of the tiny intricate w...,223441,...,Cluster,41,Lacerta,Lacerta,28,Young Stellar Object,Young Stellar Object,NaN,7,Unknown
3,Abell 80,Neb,PN,EHY/q,2.200,6000,3.8,2,HII rim with faint OIII inner region. Shape is...,223446,...,Nebula,41,Lacerta,Lacerta,18,Planetary Nebula,Planetary Nebula,Ultra Faint (Mag 12+),0,Ultra Faint (Mag 12+)
4,Barnard 370,Neb,Dark,0,45.000,u,u,0,Very faint - Region of narrow dark lanes,223450,...,Nebula,17,Cepheus,Cepheus,15,Dark Nebula,Dark Nebula,NaN,7,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3283,IAU:604,Moon,Saturnian Moon,NaN,0.042,NaN,NaN,2,Bright icy surface; visible in large scopes.,NaN,...,Moon,84,Orbit,Orbit,33,Saturnian Moon,Saturnian Moon,Ghostly (Mag 10-12),1,Ghostly (Mag 10-12)
3284,IAU:603,Moon,Saturnian Moon,NaN,0.036,NaN,NaN,2,Small and faint; requires dark skies.,NaN,...,Moon,84,Orbit,Orbit,33,Saturnian Moon,Saturnian Moon,Ghostly (Mag 10-12),1,Ghostly (Mag 10-12)
3285,IAU:602,Moon,Saturnian Moon,NaN,0.030,NaN,NaN,2,Bright icy moon with geysers; very faint.,NaN,...,Moon,84,Orbit,Orbit,33,Saturnian Moon,Saturnian Moon,Ghostly (Mag 10-12),1,Ghostly (Mag 10-12)
3286,IAU:601,Moon,Saturnian Moon,NaN,0.024,NaN,NaN,1,Small Saturnian moon with a large crater; fain...,NaN,...,Moon,84,Orbit,Orbit,33,Saturnian Moon,Saturnian Moon,Ultra Faint (Mag 12+),0,Ultra Faint (Mag 12+)


In [16]:
bins = [    0,      0.5,      1,      2,      5,       10,       30,        100,      3000]
labels = ['Very-Tiny (<0.5′)', 'Tiny (0.5–1′)', 'Small (1–2′)', 'Compact (2–5′)', 'Moderate (5–10′)', 'Prominent (10–30′)', 'Extended (30–100′)', 'Expansive (100′+)']
data['SizeBin'] = pd.cut(data['Size'], bins=bins, labels=labels, include_lowest=True)

labels = labels + ['Unknown']
Sz_enum = {abbr: idx for idx, abbr in enumerate(labels)}
Sz_lookup = {idx: name for idx, name in enumerate(labels)}
data['Sz'] = data['SizeBin'].map(Sz_enum).astype('Int64')
unknown_idx = Sz_enum['Unknown']
data['Sz'] = data['Sz'].fillna(unknown_idx)
data['SzName'] = data['Sz'].map(Sz_lookup)


# Step 2: Group by SizeBin and Type
grouped = data.groupby(['Sz', 'Type'], observed=True).size().reset_index(name='Count')

# Step 3: Plot with color by Type
fig = px.bar(grouped, x='Sz', y='Count', color='Type',
             title='Object Size Distribution by Type (Arcmin)',
             category_orders={'SizeBin': labels})  # ensures correct bin order

fig.show()
print(Sz_lookup)


{0: 'Very-Tiny (<0.5′)', 1: 'Tiny (0.5–1′)', 2: 'Small (1–2′)', 3: 'Compact (2–5′)', 4: 'Moderate (5–10′)', 5: 'Prominent (10–30′)', 6: 'Extended (30–100′)', 7: 'Expansive (100′+)', 8: 'Unknown'}


In [17]:
Rt_lookup = {
    5: 'Showcase (Top 2%)',
    4: 'Excellent (Top 10%)',
    3: 'Good (Top 25%)',
    2: 'Typical',
    1: 'Challenging',
    0: 'Not recommended'
}
data['Rt'] = data['Rating']
data['RtName'] = data['Rt'].map(Rt_lookup)
grouped = data.groupby(['Rating', 'Type'], observed=True).size().reset_index(name='Count')
fig = px.bar(grouped, x='Rating', y='Count', color='Type', title='Object Rating by Type' ) 
fig.show()


In [18]:
import re
catalogs = [
    ('HD', 'HD '),
    ('Messier', 'M'),
    ('Caldwell', 'Caldwell '),
    ('NGC', 'NGC '),
    ('IC', 'IC '),
    ('AbellG', 'Abell '),
    ('AbellN', 'Abell '),
    ('Arp', 'Arp '),
    ('H400', 'H'),
    ('UGC', 'UGC '),
    ('PGC', 'PGC '),
    ('Hickson', 'Hickson '),
    ('Griff.', 'Griffiths '),
    ('Kohou.', 'Kohoutek '),
    ('Mink.', 'Minkowski '),
    ('Barn.', 'Barnard '),
    ('Sh2', 'Sh 2-'),
    ('LBN', 'LBN '),
    ('RCW', 'RCW '),
    ('vdB', 'vdB '),
    ('LDN', 'LDN '),
    ('SNR', 'SNR '),
    ('Gum', 'Gum '),
    ('HT', 'HT '),
    ('SD', 'SD '),
    ('OB', 'OB '),
    ('SP', 'SP '),
    ('FG', 'FG '),
    ('Name', ''),
    ('Object', ''),
]

import re
from collections import OrderedDict

def normalize_catalog_id(text):
    # Remove leading zeros from catalog numbers (e.g. "Abell 01" → "Abell 1")
    text = re.sub(r'(\D+)\s*0+(\d+)', r'\1 \2', str(text))
    # Remove trailing patterns like " (3 of 5)"
    text = re.sub(r'\s*\(\s*\d+\s*of\s*\d+\s*\)', '', text)
    # Rename AbellG to Abell"
    text = re.sub(r'^AbellG\b', 'Abell', text)
    # Remove space in Messier eg M 23 to M23
    text = re.sub(r'^M\s*(\d+)', r'M\1', text)
    # Rename Sh2 to Sh 2
    text = re.sub(r'^Sh2', r'Sh 2', text)
    # Single spaces
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def get_catalog_ids(row):
    primary = None
    nickname = None
    secondary = []
    for col, prefix in catalogs:
        val = row.get(col)
        if pd.notnull(val):
            formatted = f"{prefix}{val}"
            normalized = normalize_catalog_id(formatted)
            if not primary:
                primary = normalized
            else:
                if col=='Name': 
                    nickname = normalized
                elif col!='Object':
                    secondary.append(normalized)
                else:
                    if not (normalized==nickname or normalized==primary):
                        secondary.append(normalized)

    unique_secondary = list(OrderedDict.fromkeys(secondary))
    return pd.Series({ 
        'MainID': primary, 
        'OtherIDs': ', '.join(unique_secondary) if unique_secondary else None,
        'Name': nickname,
    })

data[['MainID', 'OtherIDs', 'Name']] = data.apply(get_catalog_ids, axis=1)

In [19]:
# Suppliment data
data.loc[data['OtherIDs'] == 'SMC', 'Name'] = 'Small Magellanic Cloud'
data.loc[data['MainID'] == 'LMC', 'Name'] = 'Large Magellanic Cloud'
data.loc[data['MainID'] == 'NGC 2327', 'Name'] = 'Seagull Nebula'
data.loc[data['MainID'] == 'Dark Doodad', 'Name'] = 'Dark Doodad'
data.loc[data['MainID'] == 'Dark Doodad', 'MainID'] = 'NGC 4372'
data.loc[data['MainID'] == 'Caldwell 106', 'Name'] = '47 Tucanae'
data.loc[data['MainID'] == 'vdB 21', 'Name'] = 'Maia Nebula'
data.loc[data['MainID'] == 'vdB 22', 'Name'] = 'Merope Nebula'
data.loc[data['MainID'] == 'vdB 23', 'Name'] = 'Pleiades Nebula'
data.loc[data['MainID'] == 'Caldwell 50', 'Name'] = 'Satellite Cluster'
data.loc[data['MainID'] == 'Caldwell 77', 'Name'] = 'Centaurus A'
data.loc[data['MainID'] == 'NGC 7822', 'Name'] = 'Cosmic Question Mark'
data.loc[data['MainID'] == 'Chamaeleon I', 'Name'] = 'Chamaeleon dark cloud'
data.loc[data['OtherIDs'] == 'Chamaeleon II', 'Name'] = 'Haast Eagle Nebula, Possum Nebula'
data.loc[data['OtherIDs'] == 'Chamaeleon II', 'MainID'] = 'Chamaeleon II'
data.loc[data['OtherIDs'] == 'Chamaeleon II', 'OtherIDs'] = None
data.loc[data['MainID'] == 'Chamaeleon III', 'Name'] = 'Chamaeleon dark cloud'
data.loc[data['MainID'] == 'LBN 468', 'Name'] = 'Gyulbudaghian\'s Nebula'
data.loc[data['MainID'] == 'LDN 1251', 'Name'] = 'Rotten Fish Nebula'
data.loc[data['MainID'] == 'Abell 262', 'Name'] = 'Galaxy Cluster in Andromeda'
data.loc[data['MainID'] == 'Barnard 7', 'Name'] = 'Taurus Molecular Clouds'
data.loc[data['MainID'] == 'NGC 7822', 'Name'] = 'Cosmic Question Mark'
data.loc[data['MainID'] == 'NGC 7822', 'Name'] = 'Cosmic Question Mark'
data.loc[data['MainID'] == 'NGC 7822', 'Name'] = 'Cosmic Question Mark'
data.loc[data['MainID'] == 'NGC 7822', 'Name'] = 'Cosmic Question Mark'


In [20]:
data[['MainID', 'Name', 'VzName', 'C1Name', 'C2Name', 'CnName', 'SzName', 'RtName', 'Notes','OtherIDs']]

,MainID,Name,VzName,C1Name,C2Name,CnName,SzName,RtName,Notes,OtherIDs
0,Sh 2-139,None,Unknown,Nebula,Emission Nebula,Cepheus,Prominent (10–30′),Challenging,Faint HII region with small brighter arc,LBN 502
1,NGC 7309,None,Ultra Faint (Mag 12+),Galaxy,Spiral Galaxy,Aquila,Small (1–2′),Typical,Unusual arm structure - 3 primary arms and 3 s...,PGC 69183
2,V 375 Lac,None,Unknown,Cluster,Young Stellar Object,Lacerta,Compact (2–5′),Typical,YSO that is the source of the tiny intricate w...,None
3,Abell 80,None,Ultra Faint (Mag 12+),Nebula,Planetary Nebula,Lacerta,Compact (2–5′),Typical,HII rim with faint OIII inner region. Shape is...,None
4,Barnard 370,None,Unknown,Nebula,Dark Nebula,Cepheus,Extended (30–100′),Not recommended,Very faint - Region of narrow dark lanes,None
...,...,...,...,...,...,...,...,...,...,...
3283,Dione,None,Ghostly (Mag 10-12),Moon,Saturnian Moon,Orbit,Very-Tiny (<0.5′),Typical,Bright icy surface; visible in large scopes.,IAU:604
3284,Tethys,None,Ghostly (Mag 10-12),Moon,Saturnian Moon,Orbit,Very-Tiny (<0.5′),Typical,Small and faint; requires dark skies.,IAU:603
3285,Enceladus,None,Ghostly (Mag 10-12),Moon,Saturnian Moon,Orbit,Very-Tiny (<0.5′),Typical,Bright icy moon with geysers; very faint.,IAU:602
3286,Mimas,None,Ultra Faint (Mag 12+),Moon,Saturnian Moon,Orbit,Very-Tiny (<0.5′),Challenging,Small Saturnian moon with a large crater; fain...,IAU:601


In [21]:
def write_file(filtered_df, pathname):
    sorted_df = filtered_df.sort_values(by=['Rt', 'Sz'], ascending=[False, False])
    json_str = sorted_df.to_json(orient='records', indent=2)
    print('# ', pathname, len(sorted_df))
    with open(pathname, 'w') as f:
        f.write(json_str)

import json
print('# Total Number of Objects: ', len(data))
selected_columns = ['MainID', 'Name', 'Notes','Class', 'OtherIDs', 'Rt', 'Sz', 'Vz', 'Cn', 'C1', 'C2', 'RA_hr', 'Dec_deg']
# Top 25% and (Down to Faint ie Magnitude<10 and Down to Prominant ie Size>5')
filtered_df = data[ data['C1'].isin([4,5]) | (data['Rt'].isin([5, 4, 3]) &     (data['Vz'].isin([7, 6, 5, 4, 3, 2]) &  data['Sz'].isin([8, 7, 6, 5, 4]) )) ][selected_columns]
write_file(filtered_df, '../pilot/public/catalog_top25_lg.json')
# Top 25% not included above
filtered_df = data[   data['Rt'].isin([5, 4, 3]) &    ~(data['Vz'].isin([7, 6, 5, 4, 3, 2]) &  data['Sz'].isin([8, 7, 6, 5, 4]) ) ][selected_columns]
write_file(filtered_df, '../pilot/public/catalog_top25_sm.json')
# All Typical Objects
filtered_df = data[   data['Rt'].isin([2]) ][selected_columns]
write_file(filtered_df, '../pilot/public/catalog_typical_md.json')

print('# Rt: ', Rt_lookup)
print('# Vz: ', Vz_lookup)
print('# Sz: ', Sz_lookup)
print('# C1: ', C1_lookup)
print('# C2: ', C2_lookup)
print('# Cn: ', Cn_lookup)
print()


# Total Number of Objects:  3288
#  ../pilot/public/catalog_top25_lg.json 555
#  ../pilot/public/catalog_top25_sm.json 413
#  ../pilot/public/catalog_typical_md.json 1714
# Rt:  {5: 'Showcase (Top 2%)', 4: 'Excellent (Top 10%)', 3: 'Good (Top 25%)', 2: 'Typical', 1: 'Challenging', 0: 'Not recommended'}
# Vz:  {0: 'Ultra Faint (Mag 12+)', 1: 'Ghostly (Mag 10-12)', 2: 'Faint (Mag 8-10)', 3: 'Dim (Mag 6-8)', 4: 'Visible (Mag 4-6)', 5: 'Bright (Mag 2-4)', 6: 'Brilliant (Mag <2)', 7: 'Unknown'}
# Sz:  {0: 'Very-Tiny (<0.5′)', 1: 'Tiny (0.5–1′)', 2: 'Small (1–2′)', 3: 'Compact (2–5′)', 4: 'Moderate (5–10′)', 5: 'Prominent (10–30′)', 6: 'Extended (30–100′)', 7: 'Expansive (100′+)', 8: 'Unknown'}
# C1:  {0: 'Nebula', 1: 'Galaxy', 2: 'Cluster', 3: 'Star', 4: 'Planet', 5: 'Moon'}
# C2:  {0: 'Set of Chained Galaxies', 1: 'Set of Clustered Galaxies', 2: 'Set of Grouped Galaxies', 3: 'Set of Merging Galaxies', 4: 'Pair of Galaxies', 5: 'Trio of Galaxies', 6: 'Blue Compact Dwarf Galaxy', 7: 'Collisi

In [ ]:
# Total Number of Objects:  3288
#  ../pilot/public/catalog_top25_lg.json 555
#  ../pilot/public/catalog_top25_sm.json 413
#  ../pilot/public/catalog_typical_md.json 1714
# Rt:  {5: 'Showcase (Top 2%)', 4: 'Excellent (Top 10%)', 3: 'Good (Top 25%)', 2: 'Typical', 1: 'Challenging', 0: 'Not recommended'}
# Vz:  {0: 'Ultra Faint (Mag 12+)', 1: 'Ghostly (Mag 10-12)', 2: 'Faint (Mag 8-10)', 3: 'Dim (Mag 6-8)', 4: 'Visible (Mag 4-6)', 5: 'Bright (Mag 2-4)', 6: 'Brilliant (Mag <2)', 7: 'Unknown'}
# Sz:  {0: 'Very-Tiny (<0.5′)', 1: 'Tiny (0.5–1′)', 2: 'Small (1–2′)', 3: 'Compact (2–5′)', 4: 'Moderate (5–10′)', 5: 'Prominent (10–30′)', 6: 'Extended (30–100′)', 7: 'Expansive (100′+)', 8: 'Unknown'}
# C1:  {0: 'Nebula', 1: 'Galaxy', 2: 'Cluster', 3: 'Star', 4: 'Planet', 5: 'Moon'}
# C2:  {0: 'Set of Chained Galaxies', 1: 'Set of Clustered Galaxies', 2: 'Set of Grouped Galaxies', 3: 'Set of Merging Galaxies', 4: 'Pair of Galaxies', 5: 'Trio of Galaxies', 6: 'Blue Compact Dwarf Galaxy', 7: 'Collisional Ring Galaxy', 8: 'Dwarf Galaxy', 9: 'Elliptical Galaxy', 10: 'Flocculent Galaxy', 11: 'Lenticular Galaxy', 12: 'Magellanic Galaxy', 13: 'Polar Galaxy', 14: 'Spiral Galaxy', 15: 'Dark Nebula', 16: 'Emission Nebula', 17: 'Molecular Cloud Nebula', 18: 'Planetary Nebula', 19: 'Protoplanetary Nebula', 20: 'Reflection Nebula', 21: 'Supernova Remnant Nebula', 22: 'Globular Cluster', 23: 'Herbig-Haro Object', 24: 'Nova Object', 25: 'Open Cluster', 26: 'Star', 27: 'Star Cloud', 28: 'Young Stellar Object', 29: 'Planet', 30: 'Dwarf Planet', 31: 'Martian Moon', 32: 'Galilean Moon', 33: 'Saturnian Moon', 34: 'Natural Satellite', 35: 'Space Station', 36: 'Satellite', 37: 'Rocket Body', 38: 'Space Debris', 39: 'Comet', 40: 'Asteroid'}
# Cn:  {0: 'Andromeda', 1: 'Antlia', 2: 'Apus', 3: 'Aquila', 4: 'Aquarius', 5: 'Ara', 6: 'Aries', 7: 'Auriga', 8: 'Boötes', 9: 'Canis Major', 10: 'Canis Minor', 11: 'Canes Venatici', 12: 'Camelopardalis', 13: 'Capricornus', 14: 'Carina', 15: 'Cassiopeia', 16: 'Centaurus', 17: 'Cepheus', 18: 'Cetus', 19: 'Chamaeleon', 20: 'Circinus', 21: 'Cancer', 22: 'Columba', 23: 'Coma Berenices', 24: 'Corona Australis', 25: 'Corona Borealis', 26: 'Crater', 27: 'Crux', 28: 'Corvus', 29: 'Cygnus', 30: 'Delphinus', 31: 'Dorado', 32: 'Draco', 33: 'Eridanus', 34: 'Fornax', 35: 'Gemini', 36: 'Grus', 37: 'Hercules', 38: 'Horologium', 39: 'Hydra', 40: 'Leo Minor', 41: 'Lacerta', 42: 'Leo', 43: 'Lepus', 44: 'Libra', 45: 'Lupus', 46: 'Lynx', 47: 'Lyra', 48: 'Mensa', 49: 'Microscopium', 50: 'Monoceros', 51: 'Musca', 52: 'Norma', 53: 'Octans', 54: 'Ophiuchus', 55: 'Orion', 56: 'Pavo', 57: 'Pegasus', 58: 'Perseus', 59: 'Phoenix', 60: 'Pictor', 61: 'Piscis Austrinus', 62: 'Pisces', 63: 'Puppis', 64: 'Pyxis', 65: 'Reticulum', 66: 'Sculptor', 67: 'Scorpius', 68: 'Scutum', 69: 'Serpens', 70: 'Sextans', 71: 'Sagitta', 72: 'Sagittarius', 73: 'Taurus', 74: 'Telescopium', 75: 'Triangulum Australe', 76: 'Triangulum', 77: 'Tucana', 78: 'Ursa Major', 79: 'Ursa Minor', 80: 'Vela', 81: 'Virgo', 82: 'Volans', 83: 'Vulpecula', 84: 'Orbit'}


In [23]:
import numpy as np
from datetime import datetime, timezone

a='░▁▂▃▄▅▆▇█▔░▁_▂▃▄▅▆▇▔'

def categorize_alt(alt):
    if alt < 0:
        return 'Below Horizon'  # Not visible
    elif alt < 12:
        return 'Near Horizon'   # Rising or setting, poor visibility
    elif alt < 30:
        return 'Low Altitude'   # Often affected by atmospheric distortion
    elif alt < 60:
        return 'Mid Altitude'   # Good visibility, moderate elevation
    elif alt < 82:
        return 'High Altitude'  # Excellent visibility, optimal for imaging
    else:
        return 'Near Zenith'    # Peak elevation, polaris cannot reach


def approx_altaz(ra_deg, dec_deg, observer_lat, observer_lon, time_utc):
    # Convert to radians
    ra = np.radians(ra_deg)
    dec = np.radians(dec_deg)
    lat = np.radians(observer_lat)

    # Julian Date
    jd = (time_utc - datetime(2000, 1, 1, tzinfo=timezone.utc)).total_seconds() / 86400.0 + 2451545.0

    # Local Sidereal Time (LST) in degrees
    lst_deg = (100.46 + 0.985647 * (jd - 2451545.0) + observer_lon + np.degrees(ra)) % 360
    ha_rad = np.radians(lst_deg - ra_deg)  # Hour angle in radians

    # Altitude
    alt_rad = np.arcsin(np.sin(lat) * np.sin(dec) + np.cos(lat) * np.cos(dec) * np.cos(ha_rad))
    alt_deg = np.degrees(alt_rad)

    # Azimuth
    cz = (np.sin(dec) - np.sin(lat) * np.sin(alt_rad)) / (np.cos(lat) * np.cos(alt_rad))
    cz = np.clip(cz, -1, 1)  # Avoid domain errors
    az_rad = np.arccos(cz)
    az_deg = np.degrees(az_rad)
    az_deg = np.where(np.sin(ha_rad) < 0, az_deg, 360 - az_deg)

    return az_deg, alt_deg
